In [2]:
import os
os.chdir('../../')

In [3]:
adult_sera = ['33C', '34C', '197C', '199C', '215C', 
              '210C', '74C', '68C', '150C', '18C']

ped_sera = [2367, 3944, 2462, 2389, 2323, 2388, 2463, 3973, 4299, 4584]

teen_sera = [2343, 2350, 2365, 2382, 3866, 2380, 3856, 3857, 3862, 3895]

In [28]:
def hill_plus_escape(serum_list, age_cohort):
    hill_coefficients = []
    sera = []  
    max_site_escapes = []
    max_aa_escapes = []
    
    for serum in serum_list:
        sera.append(serum)
        
        model = pickle.load(open(f'results/antibody_escape/{serum}.pickle', 'rb'))
        coeff_mean = model.curve_specs_df['hill_coefficient_mean'][0]
        hill_coefficients.append(coeff_mean)
        
        df = model.mut_escape_df
        df['sitewise_sum'] = df['escape_mean'].groupby(df['site']).transform('sum')
        max_site = df.loc[df['sitewise_sum'].idxmax()]['sitewise_sum']
        max_site_escapes.append(max_site)
        
        max_aa = df.loc[df['escape_mean'].idxmax()]['escape_mean']
        max_aa_escapes.append(max_aa)
    
    hill_df = pd.DataFrame({
        'serum': sera,
        'hill_coeff': hill_coefficients,
        'max_site_escape': max_site_escapes,
        'max_aa_escape': max_aa_escapes,
        'age_cohort': age_cohort}
    )
    
    return hill_df

In [29]:
adults = hill_plus_escape(adult_sera, '40-45')
teens = hill_plus_escape(teen_sera, '15-18')
ped = hill_plus_escape(ped_sera, '0-5')

hill_df = pd.concat([adults, teens, ped]).reset_index()

In [10]:
hill_plot = (
    alt.Chart()
    .encode(
        x=alt.X("age_cohort", 
                title="age cohort",
               ),
        y=alt.Y(
            "hill_coeff",
            title="mean Hill coefficient",
        ),
        xOffset='jitter:Q',
        color=alt.Color('age_cohort:N'
                       ).scale(scheme='set2'),
        detail='serum',
    ).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
    .mark_circle(size=50, opacity=0.7)
    .properties(width=500, height=200)
    # .mark_circle(opacity=0.7, size=20))
)

chart = alt.layer(
    hill_plot, data=hill_df
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
).resolve_axis(
    x='independent'
)

chart

alt.LayerChart(...)

In [11]:
hill_plot = (
    alt.Chart()
    .encode(
        x=alt.X("hill_coeff", 
                title="hill coefficient",
                scale=alt.Scale(domain=[1.2, 2.8])
               ),
        y=alt.Y(
            "max_site_escape",
            title="max summed site escape",
        ),
        color=alt.Color('age_cohort:N'
                       ).scale(scheme='set2'),
        detail='serum',
    )
    .mark_circle(size=50, opacity=0.7)
    .properties(width=300, height=300)
    # .mark_circle(opacity=0.7, size=20))
)

chart = alt.layer(
    hill_plot, data=hill_df
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
).resolve_axis(
    x='independent'
)

chart

alt.LayerChart(...)

In [32]:
hill_plot = (
    alt.Chart()
    .encode(
        x=alt.X("hill_coeff", 
                title="hill coefficient",
                scale=alt.Scale(domain=[1.2, 2.8])
               ),
        y=alt.Y(
            "max_aa_escape",
            title="max AA escape",
        ),
        color=alt.Color('age_cohort:N'
                       ).scale(scheme='set2'),
        detail='serum',
    )
    .mark_circle(size=50, opacity=0.7)
    .properties(width=300, height=300)
    # .mark_circle(opacity=0.7, size=20))
)

chart = alt.layer(
    hill_plot, data=hill_df
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
).resolve_axis(
    x='independent'
)

chart

alt.LayerChart(...)

In [34]:
hill_min = hill_df.loc[hill_df['hill_coeff'].idxmin()]['hill_coeff']

hill_max = hill_df.loc[hill_df['hill_coeff'].idxmax()]['hill_coeff']

print(f'min Hill coefficient: {hill_min.round(2)}\nmax Hill coefficient: {hill_max.round(2)}')

min Hill coefficient: 1.48
max Hill coefficient: 2.63
